<a href="https://colab.research.google.com/github/Jorgecardetegit/DiseaseClassifier/blob/main/Simple_CNN_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 1. Preprocessing

### Normalizing or Standardizing and rescaling

All the images in the dataset have the same size so it is not strictly necessary to Standardize or even Normalize the dataset, nevertheless I will normalize the images in order to converge faster during training and improve generalization. I won´t standardize it.

In addition the images have a size of 255*255 which is quite standard and suitable for the models I am going to fit so I won´t rescale the images.

### Data Augmentation
As stated in the EDA anlaysis the main problem in the dataset is the imbalance of data amount presented in each category. In order to deal with this issue I will use the method of data augmentation to artificially incraese the number of images in the categories with less images.

Is important to be carefull with this method, since it can provoke overfitting. To start with I will use the following technique with each variable:

- apple_scab: Apply mild to moderate augmentation. An increase of 20-50% in the number of images will be the starting point.

- black_rot: Will also apply mild to moderate augmentation. An increase of 20-50% in the number of images will be the starting point.

- cedar_apple_rust: Apply moderate to aggressive data augmentation to this class to increase the number of samples. I aim to at least double the number of images in this class

- healthy: No augmentation will be applied for now.

### Splitting ratios
- Train: 70%
- Validation: 15%
- Test: 15%

### Parameters definition

- IMAGE_WIDTH = 255
- IMAGE_HEIGHT = 255
- NUM_CHANNELS = 3
- BATCH_SIZE = 32
- EPOCHS = 50

### Data Augmentation

In [ ]:
IMAGE_HEIGHT = 255
IMAGE_WIDTH = 255
NUM_CHANNELS = 3
BATCH_SIZE = 32
EPOCHS = 50

In [ ]:
apple_scab_GEN =

In [ ]:
data_GEN = ImageDataGenerator(
      Valhorizontal_flip = True,
      rotation_range = 10
  )

In [ ]:
apple_scab_GEN = data_GEN.flow_from_directory(
    directory='/content/drive/MyDrive/Apple disease proyect/dataset/Apple___Apple_scab',
    target_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode="category",
    shuffle=True,
    augmentation_size = 1
    seed=42)

